<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


In [20]:
# Path
import sys
sys.path.append('../')

# Data
import pandas as pd

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [39]:
# Cargar CSVs limpios
df_generacion = pd.read_csv('./data/datosgeneración_limpios.csv',index_col=False)
df_demanda = pd.read_csv('./data/datosdemanda_limpios.csv',index_col=False)
df_datoseco=pd.read_csv('./data/datos_economicos_limpios.csv', encoding= "latin 1" ,sep=";")
df_datosdemo=pd.read_csv('./data/datos_demograficos_limpios.csv', encoding= "latin 1" ,sep=";")

In [43]:

# Reestructuración del DataFrame usando pivot para expandir todas las combinaciones de valores
df_datoseco2 = df_datoseco.pivot_table(
    index=["Comunidades y Ciudades Autónomas", "periodo"],
    columns=["Ramas de actividad", "Magnitud"],
    values="Total",
    aggfunc='first'
).reset_index()

# Aplanar los nombres de columnas para una estructura más sencilla
df_datoseco2.columns = [' '.join(col).strip() for col in df_datoseco2.columns.values]

# Mostrar el DataFrame resultante
df_datoseco2.head()


,Comunidades y Ciudades Autónomas,periodo,"A. Agricultura, ganadería, silvicultura y pesca Valor","B_E. Industrias extractivas, industria manufacturera, suministro de energía eléctrica, gas, vapor y aire acondicionado, suministro de agua, actividades de saneamiento, gestión de residuos y descontaminación Valor",C. - De las cuales: Industria manufacturera Valor,F. Construcción Valor,"G_I. Comercio al por mayor y al por menor, reparación de vehículos de motor y motocicletas, transporte y almacenamiento, hostelería Valor",Impuestos netos sobre los productos Valor,J. Información y comunicaciones Valor,K. Actividades financieras y de seguros Valor,L. Actividades inmobiliarias Valor,"M_N. Actividades profesionales, científicas y técnicas, actividades administrativas y servicios auxiliares Valor","O_Q. Administración pública y defensa, seguridad social obligatoria, educación, actividades sanitarias y de servicios sociales Valor",PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO Valor,"R_U. Actividades artísticas, recreativas y de entretenimiento, reparación de artículos de uso doméstico y otros servicios Valor",Valor añadido bruto total Valor
0,01 Andalucía,2019,9.079.829,17.232.082,11.124.305,10.489.402,34.808.451,15.346.559,2.987.256,5.282.605,19.116.983,10.317.005,32.862.994,164.929.489,7.406.323,149.582.930
1,01 Andalucía,2020,9.659.346,15.272.457,9.570.097,8.918.988,26.487.461,13.019.583,2.939.019,5.524.922,19.211.915,8.832.463,32.973.273,148.779.089,5.939.662,135.759.506
2,01 Andalucía,2021(P),10.862.905,17.856.427,11.165.317,9.256.853,31.594.093,15.623.189,3.131.279,5.554.191,19.747.665,10.046.304,34.131.785,164.003.435,6.198.744,148.380.246
3,02 Aragón,2019,2.132.072,7.429.647,6.113.573,2.188.416,7.139.503,3.540.487,682.179,1.252.990,3.702.194,2.047.045,6.515.304,38.049.621,1.419.784,34.509.134
4,02 Aragón,2020,2.299.319,6.951.050,5.676.180,1.900.039,5.860.666,3.108.475,670.609,1.311.945,3.720.903,1.929.362,6.601.287,35.521.579,1.167.924,32.413.104


In [38]:
# Reestructuración del DataFrame usando pivot para expandir todas las combinaciones de valores
df_expanded = df_datosdemo.pivot_table(
    index=["Comunidades y Ciudades Autónomas", "Año"],
    columns=["Sexo", "Españoles/Extranjeros", "Edad (3 grupos de edad)"],
    values="Total",
    aggfunc='first'
).reset_index()

# Aplanar los nombres de columnas para una estructura más sencilla
df_expanded.columns = [' '.join(col).strip() for col in df_expanded.columns.values]

# Mostrar el DataFrame resultante
df_expanded


,Comunidades y Ciudades Autónomas,Año,Ambos sexos Españoles 0-15 años,Ambos sexos Españoles 16-64 años,Ambos sexos Españoles 65 y más,Ambos sexos Españoles TOTAL EDADES,Ambos sexos Extranjeros 0-15 años,Ambos sexos Extranjeros 16-64 años,Ambos sexos Extranjeros 65 y más,Ambos sexos Extranjeros TOTAL EDADES,...,Mujeres Españoles 65 y más,Mujeres Españoles TOTAL EDADES,Mujeres Extranjeros 0-15 años,Mujeres Extranjeros 16-64 años,Mujeres Extranjeros 65 y más,Mujeres Extranjeros TOTAL EDADES,Mujeres TOTAL 0-15 años,Mujeres TOTAL 16-64 años,Mujeres TOTAL 65 y más,Mujeres TOTAL TOTAL EDADES
0,01 Andalucía,2019,1.321.150,5.070.866,1.366.669,7.758.685,94.289,487.833,73.433,655.555,...,776.595,3.943.366,45.881,241.357,36.469,323.707,687.793,2.766.216,813.064,4.267.073
1,01 Andalucía,2020,1.300.962,5.068.503,1.392.928,7.762.393,101.236,522.897,77.885,702.018,...,790.173,3.946.043,49.280,259.637,38.846,347.763,681.094,2.783.693,829.019,4.293.806
2,01 Andalucía,2021,1.276.255,5.069.374,1.414.862,7.760.491,100.546,530.839,80.531,711.916,...,801.177,3.946.187,48.960,263.644,40.277,352.881,669.036,2.788.578,841.454,4.299.068
3,02 Aragón,2019,171.412,717.021,281.559,1.169.992,25.728,119.530,4.041,149.299,...,158.979,594.593,12.436,59.182,2.386,74.004,95.864,411.368,161.365,668.597
4,02 Aragón,2020,169.137,714.297,283.909,1.167.343,27.490,129.805,4.753,162.048,...,160.254,593.386,13.262,63.881,2.806,79.949,95.554,414.721,163.060,673.335
5,02 Aragón,2021,166.353,713.059,284.500,1.163.912,27.585,129.661,5.103,162.349,...,160.250,591.494,13.333,63.207,2.979,79.519,94.196,413.588,163.229,671.013
6,"03 Asturias, Principado de",2019,113.729,606.460,260.985,981.174,5.608,34.010,2.008,41.626,...,152.170,512.373,2.737,18.401,1.152,22.290,58.026,323.315,153.322,534.663
7,"03 Asturias, Principado de",2020,111.475,597.761,264.227,973.463,6.100,36.881,2.340,45.321,...,153.751,508.444,2.942,19.978,1.354,24.274,57.119,320.494,155.105,532.718
8,"03 Asturias, Principado de",2021,109.099,590.890,266.358,966.347,6.013,36.951,2.481,45.445,...,154.774,504.688,2.934,20.065,1.440,24.439,55.925,316.988,156.214,529.127
9,"04 Balears, Illes",2019,155.300,625.467,163.706,944.473,29.723,160.326,14.938,204.987,...,91.184,474.922,14.560,79.150,8.071,101.781,89.915,387.533,99.255,576.703


In [32]:
print(df_datosdemo)

      Total Nacional Comunidades y Ciudades Autónomas Edad (3 grupos de edad)  \
0     Total Nacional                              NaN            TOTAL EDADES   
1     Total Nacional                              NaN            TOTAL EDADES   
2     Total Nacional                              NaN            TOTAL EDADES   
3     Total Nacional                              NaN            TOTAL EDADES   
4     Total Nacional                              NaN            TOTAL EDADES   
...              ...                              ...                     ...   
2875  Total Nacional                       19 Melilla                65 y más   
2876  Total Nacional                       19 Melilla                65 y más   
2877  Total Nacional                       19 Melilla                65 y más   
2878  Total Nacional                       19 Melilla                65 y más   
2879  Total Nacional                       19 Melilla                65 y más   

             Sexo Españoles

In [22]:
# Conexion con la base de datos
try:
    conexioncomunidades=psycopg2.connect(
    database="comunidades",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [23]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

<cursor object at 0x000001DDC72C7680; closed: 0>

In [24]:
# Creación de tablas

querydecreacion1 = """
    CREATE TABLE IF NOT EXISTS comunidades (
    ID_Comunidad INT PRIMARY KEY,
    nombre_comunidad VARCHAR(255) NOT NULL
);"""
cursorcomunidades.execute(querydecreacion1)
conexioncomunidades.commit()

querydecreacion2 = """
    CREATE TABLE IF NOT EXISTS datos_generacion (
    id SERIAL PRIMARY KEY,
    valor_generacion FLOAT,
    porcentaje_generacion FLOAT,
    fecha DATE,
    tipo VARCHAR(50),
    ID_Comunidad INT,
    FOREIGN KEY (ID_Comunidad) REFERENCES comunidades(ID_Comunidad)
);
"""
cursorcomunidades.execute(querydecreacion2)
conexioncomunidades.commit()

querydecreacion3 = """
    CREATE TABLE IF NOT EXISTS datos_demanda (
    id SERIAL PRIMARY KEY,
    valor_demanda FLOAT,
    porcentaje_demanda FLOAT,
    fecha DATE,
    tipo VARCHAR(50),
    ID_Comunidad INT,
    FOREIGN KEY (ID_Comunidad) REFERENCES comunidades(ID_Comunidad)
);"""
cursorcomunidades.execute(querydecreacion3)
conexioncomunidades.commit()

querydecreacion4 = """
    CREATE TABLE IF NOT EXISTS datos_demograficos (
    id SERIAL PRIMARY KEY,
    region VARCHAR(255),
    comunidad VARCHAR(255),
    grupo_edad VARCHAR(255),
    nacionalidad VARCHAR(50),
    sexo VARCHAR(10),
    año INT,
    poblacion_total FLOAT,
    ID_Comunidad INT,
    FOREIGN KEY (ID_Comunidad) REFERENCES comunidades(ID_Comunidad)
);"""
cursorcomunidades.execute(querydecreacion4)
conexioncomunidades.commit()

querydecreacion5 = """
    CREATE TABLE IF NOT EXISTS datos_economicos (
    id SERIAL PRIMARY KEY,
    region VARCHAR(255),
    comunidad VARCHAR(255),
    sector_actividad VARCHAR(255),
    magnitud VARCHAR(50),
    periodo INT,
    valor_total FLOAT,
    ID_Comunidad INT,
    FOREIGN KEY (ID_Comunidad) REFERENCES comunidades(ID_Comunidad)
);
"""
cursorcomunidades.execute(querydecreacion5)
conexioncomunidades.commit()

In [25]:
# Diccionario de códigos de comunidad
cod_comunidades = {
    'Ceuta': 8744,
    'Melilla': 8745,
    'Andalucía': 4,
    'Aragón': 5,
    'Cantabria': 6,
    'Castilla - La Mancha': 7,
    'Castilla y León': 8,
    'Cataluña': 9,
    'País Vasco': 10,
    'Principado de Asturias': 11,
    'Comunidad de Madrid': 13,
    'Comunidad Foral de Navarra': 14,
    'Comunitat Valenciana': 15,
    'Extremadura': 16,
    'Galicia': 17,
    'Illes Balears': 8743,
    'Canarias': 8742,
    'Región de Murcia': 21,
    'La Rioja': 20
}

In [26]:
# Mapear nombres alternativos a los nombres del diccionario
nombres_alternativos = {
    'Asturias, Principado de': 'Principado de Asturias',
    'Rioja, La': 'La Rioja',
    'Balears, Illes': 'Illes Balears',
    'Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra'
}

# Renombrar columnas para coincidir con los nombres esperados
expected_columns = {
    'Total Nacional': 'region',
    'Comunidades y Ciudades Autónomas': 'comunidad',
    'Edad (3 grupos de edad)': 'grupo_edad',
    'Españoles/Extranjeros': 'nacionalidad',
    'Sexo': 'sexo',
    'Año': 'año',
    'Total': 'poblacion_total'
}
df_datosdemo.rename(columns=expected_columns, inplace=True)

# Convertir año y población total a tipos adecuados
df_datosdemo['año'] = pd.to_numeric(df_datosdemo['año'], errors='coerce')
df_datosdemo['poblacion_total'] = pd.to_numeric(df_datosdemo['poblacion_total'].str.replace('.', '', regex=False), errors='coerce')

# Eliminar filas con valores nulos en `poblacion_total` o `comunidad`
df_datosdemo = df_datosdemo.dropna(subset=['poblacion_total', 'comunidad'])

# Limpiar nombres de comunidades y reemplazar nombres alternativos
df_datosdemo['comunidad'] = df_datosdemo['comunidad'].str.replace(r'^\d+\s+', '', regex=True)  # Eliminar prefijos numéricos
df_datosdemo['comunidad'] = df_datosdemo['comunidad'].replace(nombres_alternativos)  # Reemplazar nombres alternativos

# Asignar `ID_Comunidad` usando el diccionario
df_datosdemo['ID_Comunidad'] = df_datosdemo['comunidad'].map(cod_comunidades)

# Convertir el DataFrame limpio en una lista de tuplas para la inserción en SQL (incluyendo ID_Comunidad)
valores_datos_demograficos = [tuple(x) for x in df_datosdemo[['region', 'comunidad', 'grupo_edad', 'nacionalidad', 'sexo', 'año', 'poblacion_total', 'ID_Comunidad']].values]

In [27]:
# Crear el DataFrame de comunidades a partir del diccionario
df_comunidades = pd.DataFrame(list(cod_comunidades.items()), columns=['nombre_comunidad', 'ID_Comunidad'])

# Añadir ID_Comunidad en los DataFrames usando el diccionario
df_datosdemo['ID_Comunidad'] = df_datosdemo['Comunidades y Ciudades Autónomas'].map(cod_comunidades)
df_datoseco['ID_Comunidad'] = df_datoseco['Comunidades y Ciudades Autónomas'].map(cod_comunidades)

# Extraer listas de valores para SQL
# Valores para la tabla comunidades
valores_comunidades = [tuple(x) for x in df_comunidades[['ID_Comunidad', 'nombre_comunidad']].values]

# Valores para la tabla datos_demanda
valores_datos_demanda = [tuple(x) for x in df_demanda[['value', 'percentage', 'datetime', 'Tipo', 'Id_Comunidad']].values]

# Valores para la tabla datos_generacion
valores_datos_generacion = [tuple(x) for x in df_generacion[['value', 'percentage', 'datetime', 'Tipo', 'Id_Comunidad']].values]


KeyError: 'Comunidades y Ciudades Autónomas'

In [ ]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

<cursor object at 0x0000029503A15B60; closed: 0>

In [ ]:
queryinsercion = 'INSERT INTO comunidades (ID_Comunidad, nombre_comunidad) VALUES (%s,%s);'
listavaloresinsercion=valores_comunidades
cursorcomunidades.executemany(queryinsercion,listavaloresinsercion)
conexioncomunidades.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «comunidades_pkey»
DETAIL:  Ya existe la llave (id_comunidad)=(8744).


In [ ]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

<cursor object at 0x000001DDC72C4D60; closed: 0>

In [ ]:
queryinsercion = 'INSERT INTO datos_demograficos (region, comunidad, grupo_edad, nacionalidad, sexo, año, poblacion_total, ID_Comunidad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);'
listavaloresinsercion=valores_datos_demograficos
cursorcomunidades.executemany(queryinsercion,listavaloresinsercion)
conexioncomunidades.commit()

TypeError: not all arguments converted during string formatting

In [ ]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

In [ ]:
queryinsercion = 'INSERT INTO datos_economicos (region, comunidad, sector_actividad, magnitud, periodo, valor_total) VALUES (%s,%s,%s,%s,%s,%s);'
listavaloresinsercion=valores_datos_economicos
cursorcomunidades.executemany(queryinsercion,listavaloresinsercion)
conexioncomunidades.commit()

TypeError: not all arguments converted during string formatting

In [ ]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

In [ ]:
queryinsercion = 'INSERT INTO datos_demanda (valor_demanda, porcentaje_demanda, fecha, tipo, ID_Comunidad) VALUES (%s,%s,%s,%s,%s);'
listavaloresinsercion=valores_datos_demanda
cursorcomunidades.executemany(queryinsercion,listavaloresinsercion)
conexioncomunidades.commit()

In [ ]:
# Primero, deshacer la transacción fallida
conexioncomunidades.rollback()

cursorcomunidades = conexioncomunidades.cursor()
cursorcomunidades

In [ ]:
queryinsercion = 'INSERT INTO datos_generacion (valor_generacion, porcentaje_generacion, fecha, tipo, ID_Comunidad) VALUES (%s,%s,%s,%s,%s);'
listavaloresinsercion=valores_datos_generacion
cursorcomunidades.executemany(queryinsercion,listavaloresinsercion)
conexioncomunidades.commit()